In [1]:
import os
import pandas as pd
os.chdir("/Users/magnus/Documents/02_PhD/08_Other_projects/06_Kaggle_projects/ftwue")
%load_ext kedro.ipython

[10/30/24 10:49:53] INFO     Using 'conf/logging.yml' as logging configuration. You can change this ]8;id=275527;file:///Users/magnus/miniforge3/envs/ftwue/lib/python3.11/site-packages/kedro/framework/project/__init__.py\__init__.py]8;;\:]8;id=485322;file:///Users/magnus/miniforge3/envs/ftwue/lib/python3.11/site-packages/kedro/framework/project/__init__.py#270\270]8;;\
                             by setting the KEDRO_LOGGING_CONFIG environment variable accordingly.                 

                    INFO     Registered line magic '%reload_kedro'                                   ]8;id=592893;file:///Users/magnus/miniforge3/envs/ftwue/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=664268;file:///Users/magnus/miniforge3/envs/ftwue/lib/python3.11/site-packages/kedro/ipython/__init__.py#61\61]8;;\

                    INFO     Registered line magic '%load_node'                                      ]8;id=873951;file:///Users/magnus/miniforge3/envs/ftwue/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=544312;file:///Users/magnus/miniforge3/envs/ftwue/lib/python3.11/site-packages/kedro/ipython/__init__.py#63\63]8;;\

                    INFO     Resolved project path as:                                              ]8;id=453213;file:///Users/magnus/miniforge3/envs/ftwue/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=600574;file:///Users/magnus/miniforge3/envs/ftwue/lib/python3.11/site-packages/kedro/ipython/__init__.py#178\178]8;;\
                             /Users/magnus/Documents/02_PhD/08_Other_projects/06_Kaggle_projects/ft                
                             wue.                                                                                  
                             To set a different path, run '%reload_kedro <project_root>'                           

                    INFO     Setting up tables in database                                        ]8;id=343612;file:///Users/magnus/Documents/02_PhD/08_Other_projects/06_Kaggle_projects/ftwue/src/ftwue/datasets/sqldatasets.py\sqldatasets.py]8;;\:]8;id=239409;file:///Users/magnus/Documents/02_PhD/08_Other_projects/06_Kaggle_projects/ftwue/src/ftwue/datasets/sqldatasets.py#41\41]8;;\

                    INFO     Kedro is sending anonymous usage data with the sole purpose of improving ]8;id=828783;file:///Users/magnus/miniforge3/envs/ftwue/lib/python3.11/site-packages/kedro_telemetry/plugin.py\plugin.py]8;;\:]8;id=460609;file:///Users/magnus/miniforge3/envs/ftwue/lib/python3.11/site-packages/kedro_telemetry/plugin.py#233\233]8;;\
                             the product. No personal data or IP addresses are stored on our side. If              
                             you want to opt out, set the `KEDRO_DISABLE_TELEMETRY` or `DO_NOT_TRACK`              
                             environment variables, or create a `.telemetry` file in the current                   
                             working directory with the contents `consent: false`. Read more at                    
                             https://docs.kedro.org/en/stable/configuration/telemetry.html                         

[10/30/24 10:49:54] INFO     Kedro project ftwue                                                    ]8;id=488917;file:///Users/magnus/miniforge3/envs/ftwue/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=934792;file:///Users/magnus/miniforge3/envs/ftwue/lib/python3.11/site-packages/kedro/ipython/__init__.py#144\144]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=591793;file:///Users/magnus/miniforge3/envs/ftwue/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=87215;file:///Users/magnus/miniforge3/envs/ftwue/lib/python3.11/site-packages/kedro/ipython/__init__.py#145\145]8;;\
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        ]8;id=188358;file:///Users/magnus/miniforge3/envs/ftwue/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=393904;file:///Users/magnus/miniforge3/envs/ftwue/lib/python3.11/site-packages/kedro/ipython/__init__.py#151\151]8;;\

In [27]:

# this will not work anymore since combined dataset now is at later stage
run_dataset = catalog.load("combined_dataset").copy()

[10/30/24 10:56:26] INFO     Loading data from combined_dataset (ParquetDataset)...             ]8;id=469959;file:///Users/magnus/miniforge3/envs/ftwue/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=875065;file:///Users/magnus/miniforge3/envs/ftwue/lib/python3.11/site-packages/kedro/io/data_catalog.py#389\389]8;;\

In [28]:
def melt_df(df, columns_to_melt):
    
    # determine columns to melt
    value_vars = columns_to_melt
    id_vars = [col for col in run_dataset.columns if col not in value_vars]

    # perform melting
    df_melted = pd.melt(
    run_dataset,
    id_vars=id_vars,
    value_vars=value_vars,
    var_name = "traffic_type",
    value_name = "pedestrian_count",
    
    )

    # rename columns
    df_melted['traffic_type'] = df_melted['traffic_type'].replace({
        'n_pedestrians': 'total',
        'n_pedestrians_towards': 'towards',
        'n_pedestrians_away': 'away'
    })

    # sort by row_id
    df_melted = df_melted.sort_values(by=["row_id", "traffic_type"])

    # delete old id column
    df_melted.drop(columns=["id"], inplace=True)

    return df_melted

def melt_pedestrian_dataset(df):
    return melt_df(df, ["n_pedestrians", "n_pedestrians_towards", "n_pedestrians_away"])

df_melted = melt_pedestrian_dataset(run_dataset)

In [29]:
df_melted.head(3)

,row_id,streetname,incidents,collection_type,full_date,temperature,weather_condition,date,holiday,weekday,month,year,calendar_week,quarter,hour,traffic_type,pedestrian_count
233238,Kaiserstrasse_2022-09-29_0,Kaiserstrasse,no_incident,measured,2022-09-29,8,cloudy,2022-09-29,False,3,9,2022,39,3,0,away,46.0
65580,Kaiserstrasse_2022-09-29_0,Kaiserstrasse,no_incident,measured,2022-09-29,8,cloudy,2022-09-29,False,3,9,2022,39,3,0,total,57.0
149409,Kaiserstrasse_2022-09-29_0,Kaiserstrasse,no_incident,measured,2022-09-29,8,cloudy,2022-09-29,False,3,9,2022,39,3,0,towards,11.0


In [30]:
def encode_categorical_features(df):

    features_on_hot_encoding = [
        # covariates
        "incidents",
        "collection_type",
        "weather_condition",
        "weekday",
        "month",
        "calendar_week",
        "quarter",
        "hour",
        
        # target categories
        "streetname",
        "traffic_type"
    ]

    df = pd.get_dummies(df, columns=features_on_hot_encoding)
    return df
df_melted = encode_categorical_features(df_melted)

In [32]:
def remove_date(df):
    return df.drop(columns=["date"])
df_melted = remove_date(df_melted)

In [33]:
df_melted.head(3)

,row_id,full_date,temperature,holiday,year,pedestrian_count,incidents_construction_site,incidents_laser_failure,incidents_no_incident,collection_type_measured,...,hour_20,hour_21,hour_22,hour_23,streetname_Kaiserstrasse,streetname_Schoenbornstrasse,streetname_Spiegelstrasse,traffic_type_away,traffic_type_total,traffic_type_towards
233238,Kaiserstrasse_2022-09-29_0,2022-09-29,8,False,2022,46.0,False,False,True,True,...,False,False,False,False,True,False,False,True,False,False
65580,Kaiserstrasse_2022-09-29_0,2022-09-29,8,False,2022,57.0,False,False,True,True,...,False,False,False,False,True,False,False,False,True,False
149409,Kaiserstrasse_2022-09-29_0,2022-09-29,8,False,2022,11.0,False,False,True,True,...,False,False,False,False,True,False,False,False,False,True


,id,row_id,streetname,n_pedestrians,n_pedestrians_towards,n_pedestrians_away,incidents,collection_type,full_date,temperature,weather_condition,date,holiday,weekday,month,year,calendar_week,quarter,hour
0,13,Schoenbornstrasse_2019-04-03_6,Schoenbornstrasse,311.0,171.0,140.0,no_incident,measured,2019-04-03 06:00:00,9,partly-cloudy-night,2019-04-03,False,2,4,2019,14,2,6
1,14,Schoenbornstrasse_2019-04-03_7,Schoenbornstrasse,324.0,182.0,142.0,no_incident,measured,2019-04-03 07:00:00,10,cloudy,2019-04-03,False,2,4,2019,14,2,7
2,15,Schoenbornstrasse_2019-04-03_8,Schoenbornstrasse,532.0,287.0,245.0,no_incident,measured,2019-04-03 08:00:00,10,cloudy,2019-04-03,False,2,4,2019,14,2,8
3,16,Schoenbornstrasse_2019-04-03_9,Schoenbornstrasse,1153.0,592.0,561.0,no_incident,measured,2019-04-03 09:00:00,9,cloudy,2019-04-03,False,2,4,2019,14,2,9
4,17,Schoenbornstrasse_2019-04-03_10,Schoenbornstrasse,1769.0,901.0,868.0,no_incident,measured,2019-04-03 10:00:00,10,cloudy,2019-04-03,False,2,4,2019,14,2,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83824,83825,Kaiserstrasse_2024-09-29_19,Kaiserstrasse,NaN,NaN,NaN,no_incident,measured,2024-09-29 19:00:00,11,clear-day,2024-09-29,False,6,9,2024,39,3,19
83825,83826,Kaiserstrasse_2024-09-29_20,Kaiserstrasse,NaN,NaN,NaN,no_incident,measured,2024-09-29 20:00:00,10,clear-night,2024-09-29,False,6,9,2024,39,3,20
83826,83827,Kaiserstrasse_2024-09-29_21,Kaiserstrasse,NaN,NaN,NaN,no_incident,measured,2024-09-29 21:00:00,10,partly-cloudy-night,2024-09-29,False,6,9,2024,39,3,21
83827,83828,Kaiserstrasse_2024-09-29_22,Kaiserstrasse,NaN,NaN,NaN,no_incident,measured,2024-09-29 22:00:00,9,cloudy,2024-09-29,False,6,9,2024,39,3,22
